In [1]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [2]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/third-experiments/domain_adaptation_project/mlm', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/third-experiments/domain_adaptation_project/modules']


2024-06-05 00:46:48.329862: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-05 00:46:48.528777: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-05 00:46:49.396261: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from datasets import load_from_disk,concatenate_datasets


adapter_name="mlm_adapter_F"
dataset = load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/datasets")



In [4]:
reload(config)

    
filtered_target = dataset['train'].filter(lambda example: example['genre'] == "fiction").shuffle(seed=42)
#filtered_target = shuffled_filtered_target.train_test_split(test_size=0.1)


filtered_test_target = dataset['validation_matched'].filter(lambda example: example['genre'] == "fiction")
train_target = filtered_target
test_target = filtered_test_target
train_target[1]
 

{'promptID': 103247,
 'pairID': '103247c',
 'premise': 'Do you want to see them?"  ',
 'premise_binary_parse': "( ( ( Do ( you ( want ( to ( see them ) ) ) ) ) ? ) '' )",
 'premise_parse': "(ROOT (FRAG (S (VP (VBP Do) (SBAR (S (NP (PRP you)) (VP (VBP want) (S (VP (TO to) (VP (VB see) (NP (PRP them)))))))))) (. ?) ('' '')))",
 'hypothesis': "You can't touch them.",
 'hypothesis_binary_parse': "( You ( ( ( ca n't ) ( touch them ) ) . ) )",
 'hypothesis_parse': "(ROOT (S (NP (PRP You)) (VP (MD ca) (RB n't) (VP (VB touch) (NP (PRP them)))) (. .)))",
 'genre': 'fiction',
 'label': 2}

In [5]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoModelForMaskedLM.from_pretrained(
    config.Config.MODEL_NAME,
)
init(model)
reload(fn)
fn.print_trainable_parameters(model)

/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFor

trainable params: 109514298 || all params: 109514298 || trainable%: 100.0


In [6]:
from adapters import ConfigUnion, LoRAConfig, PrefixTuningConfig, SeqBnConfig,SeqBnInvConfig,AdapterConfig,LoRAConfig


adapter_config = SeqBnConfig(reduction_factor=2)

model.add_adapter(adapter_name, config=adapter_config)

model.train_adapter([adapter_name])
model.active_adapters = adapter_name
print(model.adapter_summary())
fn.print_trainable_parameters(model)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
mlm_adapter_F            bottleneck        7,091,712       6.513       1       1
--------------------------------------------------------------------------------
Full model                               108,891,648     100.000               0
trainable params: 7683840 || all params: 116606010 || trainable%: 6.589574585392296


In [7]:
from transformers import AutoTokenizer

reload(processed)
tokenizer = AutoTokenizer.from_pretrained(config.Config.TOKENIZER_NAME)

tokenized_dataset= processed.tokenize_dataset(train_target,tokenizer)
tokenized_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
    num_rows: 77348
})

In [8]:
chunk_size = 128
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset[444:470]

results = fn.group_texts(tokenized_samples, chunk_size)
for chunk in results["labels"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'


In [9]:
# we group texts and chunk them
lm_datasets = tokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
lm_datasets

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 17714
})

In [10]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
samples = [lm_datasets[i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")
dd=data_collator(samples)
for chunk in dd["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] have you [MASK] reason to believe that your mistress was [MASK] to take a sleeping powder last [MASK]? " [SEP] your mistress was awake all night. [SEP] [CLS] do you want to see them [MASK] " [SEP] you can'[MASK] touch them. [SEP] [CLS]'wherever your leader is,'white said. [SEP] white said that was wherever your leader is. [SEP] [CLS] [MASK] the meantime, the young adventurers [MASK] sitting bolt upright [MASK] very stiff and ill at avenues, in a taxi which, with a singular lack of originality, was also returning to the ritz via regent's park. [SEP] [MASK] adventurer [MASK] walked home instead of [MASK] [MASK] cab'

'>>> . [SEP] [CLS] my brother lawrence is convinced that we are making [MASK] fuss over nothing. [SEP] my brother [MASK] [MASK] when he [MASK] we [MASK] making a fuss. [SEP] [CLS] at [MASK] o'clock. tu [MASK]nce made up her mind. [SEP] tuppence made a decision at eleven [MASK]'clock. [SEP] [CLS] a moment later he uttered fucking [MASK]. [SEP] ten years later, he 

In [11]:

downsampled_dataset = lm_datasets.train_test_split(
    test_size=0.1, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 15942
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1772
    })
})

In [12]:

reload(fn)
trainer = fn.train_mlm_model(model,adapter_name,data_collator,tokenizer, downsampled_dataset['train'],downsampled_dataset['test'])

/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [13]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/56 [00:00<?, ?it/s]

>>> Perplexity: 14.39


In [14]:
trainer.train()

  0%|          | 0/9980 [00:00<?, ?it/s]

{'loss': 2.2243, 'grad_norm': 1.8203213214874268, 'learning_rate': 9.960000000000001e-05, 'epoch': 1.0}


  0%|          | 0/56 [00:00<?, ?it/s]

{'eval_loss': 1.8570619821548462, 'eval_runtime': 7.2411, 'eval_samples_per_second': 244.713, 'eval_steps_per_second': 7.734, 'epoch': 1.0}
{'loss': 1.9225, 'grad_norm': 1.6242866516113281, 'learning_rate': 9.476793248945148e-05, 'epoch': 2.0}


  0%|          | 0/56 [00:00<?, ?it/s]

...

  0%|          | 0/56 [00:00<?, ?it/s]

{'eval_loss': 1.5256216526031494, 'eval_runtime': 5.1566, 'eval_samples_per_second': 343.639, 'eval_steps_per_second': 10.86, 'epoch': 18.0}


Overwriting existing adapter 'mlm_adapter_F'.


{'train_runtime': 2380.5972, 'train_samples_per_second': 133.933, 'train_steps_per_second': 4.192, 'train_loss': 1.6921816575078596, 'epoch': 18.0}


TrainOutput(global_step=8982, training_loss=1.6921816575078596, metrics={'train_runtime': 2380.5972, 'train_samples_per_second': 133.933, 'train_steps_per_second': 4.192, 'train_loss': 1.6921816575078596, 'epoch': 18.0})

In [15]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/56 [00:00<?, ?it/s]

>>> Perplexity: 4.68


In [16]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)


: 

#### This is a report of mlm for goverment domain using whole genre as target and then splitt after tokenization with seed 42. I used computer metrix for preplexity and early stopping 
The results before:
>>> Perplexity: 14.39

The results after:
>>> Perplexity: 4.68